# FTX reader

In [4]:
from jupyter_imports import *
from utils.io_utils import *
from histfeed.ftx_history import main

end_time= datetime.now(timezone.utc)-timedelta(days=1)#.replace(minute=0,second=0,microsecond=0)
start_time= end_time-timedelta(days=60)
halflife=timedelta(days=1)
modes=[('h','high'),('c','close'),('l','low'),('v','size')]

In [6]:
weights = pd.read_csv(os.path.join(os.sep, 'home','david',"config", 'prod',"pfoptimizer",'weights_ftx_debug.csv')).sort_values(by='optimalWeight',key=lambda f: np.abs(f),ascending=False)
coins = list(weights.set_index(['name']).index.map(lambda f: f.replace('-PERP','')))
weights

,name,spot,mark,index,borrow,quote_borrow,ExpectedBenchmark,funding,previousWeight,optimalWeight,...,intShortCarry,intBorrow,intUSDborrow,E_long,E_short,E_intBorrow,E_intUSDborrow,direction,exchange,subaccount
0,BCH-PERP,127.4250,127.45000,127.346667,0.067047,0.040158,-0.000694,0.298044,-74.22640,-73.447588,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ftx,debug
1,AAVE-PERP,92.0000,92.00000,92.075120,0.011742,0.040158,-0.000366,-0.219150,0.00000,-67.880558,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ftx,debug
2,MKR-PERP,984.0000,982.00000,983.593390,0.011742,0.040158,-0.000317,-0.333108,-57.93800,-58.956410,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ftx,debug
3,LINK-PERP,7.0450,7.04950,7.045427,0.011742,0.040158,0.000162,0.131490,44.27010,43.560277,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ftx,debug
4,XRP-PERP,0.3618,0.36175,0.361640,0.026654,0.040158,-0.000207,0.026298,-20.63115,-21.340795,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ftx,debug


In [10]:
futures_df = main('get','ftx','wide')

INFO:histfeed:running histfeed args=('get', 'ftx', 'wide') kwargs={}
INFO:histfeed:Getting history...
INFO:histfeed:command returned                            1INCH-PERP_rate_funding  AAVE-PERP_rate_funding  \
2022-07-24 09:00:00+00:00                 0.087660               -0.008766   
2022-07-24 10:00:00+00:00                 0.105192               -0.306810   
2022-07-24 11:00:00+00:00                 0.061362               -0.184086   
2022-07-24 12:00:00+00:00                 0.052596               -0.157788   
2022-07-24 13:00:00+00:00                 0.070128               -0.201618   
2022-07-24 14:00:00+00:00                 0.017532               -0.280512   
2022-07-24 15:00:00+00:00                -0.008766               -0.341874   
2022-07-24 16:00:00+00:00                -0.096426               -0.192852   
2022-07-24 17:00:00+00:00                 0.131490               -0.227916   
2022-07-24 18:00:00+00:00                 0.113958               -0.359406   
2022-07-2

### rates markets

In [11]:
selected = [df for df in futures_df.columns if any(df.split('-')[0].split('_')[0]==coin for coin in (coins+['USD']))]
prices=futures_df[[c for c in selected if any(string in c for string in ['price_c'])]]

short_rates=futures_df[[c for c in selected if any(string in c for string in ['PERP_rate_c','rate_borrow'])]]
short_rates=short_rates.ewm(halflife=halflife,times=short_rates.index).mean()

borrow_size=futures_df[[c for c in selected if any(string in c for string in ['_rate_size','price_volume'])]]

for c in coins:
    long_perp_carry=short_rates[c+'-PERP_rate_c']-short_rates['USD_rate_borrow']
    short_perp_carry=-short_rates[c+'-PERP_rate_c']-short_rates[c+'_rate_borrow']
    short_rates[c+'_best_perp_carry']= pd.concat([long_perp_carry, short_perp_carry]).groupby(level=0).max()
   # long_rates[c+'_long_future_carry']=long_rates[c+'-1231_rate_c']-short_rates['USD_rate_borrow']
   # long_rates[c+'_short_future_carry']=-long_rates[c+'-1231_rate_c']-short_rates[c+'_rate_borrow']

short_rates.filter(like='_best_perp_carry').iplot()#))

In [12]:
short_rates.filter(like='PERP_rate_c').iplot()#))

In [13]:
short_rates.filter(like='borrow').iplot()#))

In [14]:
pd.concat([borrow_size],axis=0).iplot()#))

### carry directionality

In [ ]:
plotthat = futures_df[['BNB-PERP_rate_h',
       'BNB-PERP_rate_l', 'BNB-PERP_rate_c']]
plotthat.columns=['h','l','c']

In [ ]:
import plotly.graph_objects as go
import numpy as np
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=plotthat.index,#+x_rev,
    y=0.5*(plotthat['l'].values+plotthat['h'].values),
    fill='toself',
    fillcolor='rgba(0,100,80,0.2)',
    line_color='rgba(255,255,255,0)',
    showlegend=False,
    name='BNB',
))
fig.add_trace(go.Scatter(
    x=plotthat.index, y=plotthat['c'].values,
    line_color='rgb(0,100,80)',
    name='Fair',
))

fig.update_traces(mode='lines')
fig.show()

In [ ]:
pd.concat([prices,short_rates.filter(like='carry')],axis=0).iplot(secondary_y=list(prices.columns))

# stopout: show min/max of premiums

In [ ]:
end_time=datetime.now(timezone.utc)#.replace(minute=0,second=0,microsecond=0)
start_time=end_time-timedelta(hours=1)
granular_df=ftx_history(dirname='',start=start_time,end=end_time,timeframe='15s',coin_list=coins)

In [ ]:
def compute_premium(df: pd.DataFrame,mode=1) -> pd.Series:
    return (df.filter(like='/mark/'+ modes[mode][0]).iloc[:,0]/df.filter(like='/indexes/'+modes[mode][1]).iloc[:,0]-1)*10000
def compute_premium_vs_spot(df: pd.DataFrame,mode=1) -> pd.Series:
    return (df.filter(like='/mark/'+ modes[mode][0]).iloc[:,0]/df.filter(like='/price/'+modes[mode][0]).iloc[:,0]-1)*10000
def compile_list(premium_list: list) -> pd.DataFrame:
    premium=premium_list[0]
    for df in premium_list[1:]:
        premium=premium.join(df,how='outer')
    return premium[~premium.index.duplicated()]

In [ ]:
granular_df['premium']=compute_premium_vs_spot(granular_df)
prices=granular_df[[c+'/price/c' for c in coins]]
granular_df[['premium']].iplot()

In [ ]:
premium=compute_premium(granular_df,mode=0)
ewMax=premium.resample('1h').max()
ewMax=ewMax[~ewMax.index.duplicated()]
pd.concat([prices,ewMax],axis=0).iplot(secondary_y=list(prices.columns))

In [ ]:
premium=compute_premium(granular_df,mode=2)
ewMin=premium.resample('1h').min()
ewMin=ewMin[~ewMin.index.duplicated()]
pd.concat([prices,ewMin],axis=0).iplot(secondary_y=list(prices.columns))